In [5]:
from sympy import *
from sympy.simplify.fu import sincos_to_sum

In [6]:
x, y, z, eps, k, S = symbols('x y z varepsilon k S')
xi = Matrix([x, y, z])

## Generate $Q$-tensor

In [7]:
Q_mat = S * Matrix([[Rational(2, 3), eps*sin(k*x), 0],
                    [eps*sin(k*x), -Rational(1, 3), 0],
                    [0, 0, -Rational(1, 3)]])

Q_mat

Matrix([
[                2*S/3, S*varepsilon*sin(k*x),    0],
[S*varepsilon*sin(k*x),                  -S/3,    0],
[                    0,                     0, -S/3]])

## Generate elastic stress tensor and corresponding force

In [8]:
sigma_e = zeros(3)
for i in range(3):
    for j in range(3):
        sigma_e[i, j] = -sum([Q_mat[k, l].diff(xi[i]) * Q_mat[k, l].diff(xi[j])
                              for k in range(3)
                              for l in range(3)])

In [9]:
sigma_e

Matrix([
[-2*S**2*k**2*varepsilon**2*cos(k*x)**2, 0, 0],
[                                     0, 0, 0],
[                                     0, 0, 0]])

In [10]:
f_e = zeros(3, 1)
for i in range(3):
    f_e[i, 0] = sum([sigma_e[k, i].diff(xi[k])
                     for k in range(3)])
f_e = trigsimp(f_e)
f_e

Matrix([
[2*S**2*k**3*varepsilon**2*sin(2*k*x)],
[                                   0],
[                                   0]])

## Generate $\mu_2$ viscous stress tensor and corresponding force

In [11]:
Lambda1 = Function("\Lambda_1")(x)
Lambda2 = Function("\Lambda_2")(x)
Lambda4 = Function("\Lambda_4")(x)
Lambda_mat = Matrix([[Lambda1, Lambda2, 0],
                     [Lambda2, Lambda4, 0],
                     [0, 0, -(Lambda1 + Lambda4)]])
Lambda_mat

Matrix([
[\Lambda_1(x), \Lambda_2(x),                            0],
[\Lambda_2(x), \Lambda_4(x),                            0],
[           0,            0, -\Lambda_1(x) - \Lambda_4(x)]])

In [12]:
alpha = symbols('alpha')

In [13]:
lap_Q_mat = zeros(3)
for i in range(3):
    for j in range(3):
        lap_Q_mat[i, j] = sum([Q_mat[i, j].diff(xi[k]).diff(xi[k])
                              for k in range(3)])
        
h = alpha*Q_mat - Lambda_mat + lap_Q_mat
h

Matrix([
[                                             2*S*alpha/3 - \Lambda_1(x), S*alpha*varepsilon*sin(k*x) - S*k**2*varepsilon*sin(k*x) - \Lambda_2(x),                                        0],
[S*alpha*varepsilon*sin(k*x) - S*k**2*varepsilon*sin(k*x) - \Lambda_2(x),                                               -S*alpha/3 - \Lambda_4(x),                                        0],
[                                                                      0,                                                                       0, -S*alpha/3 + \Lambda_1(x) + \Lambda_4(x)]])

In [14]:
f_h = zeros(3, 1)
for i in range(3):
    f_h[i, 0] = sum([h[k, i].diff(xi[k])
                     for k in range(3)])

f_h

Matrix([
[                                                            -Derivative(\Lambda_1(x), x)],
[S*alpha*k*varepsilon*cos(k*x) - S*k**3*varepsilon*cos(k*x) - Derivative(\Lambda_2(x), x)],
[                                                                                       0]])

In [15]:
Q_vec = zeros(5, 1)
Q_vec[0, 0] = Q_mat[0, 0]
Q_vec[1, 0] = Q_mat[0, 1]
Q_vec[2, 0] = Q_mat[0, 2]
Q_vec[3, 0] = Q_mat[1, 1]
Q_vec[4, 0] = Q_mat[1, 2]

Q_vec

Matrix([
[                2*S/3],
[S*varepsilon*sin(k*x)],
[                    0],
[                 -S/3],
[                    0]])

In [16]:
Q_vec.diff(x)

Matrix([
[                      0],
[S*k*varepsilon*cos(k*x)],
[                      0],
[                      0],
[                      0]])

In [17]:
dLambda1_dQ2 = symbols('\\frac{\partial\Lambda_1}{\partial\ Q_2}')
dLambda2_dQ2 = symbols('\\frac{\partial\Lambda_2}{\partial\ Q_2}')

In [18]:
f_h = f_h.subs(Lambda1.diff(x), 
               Q_vec[1].diff(x) * dLambda1_dQ2)
f_h = f_h.subs(Lambda2.diff(x), 
               Q_vec[1].diff(x) * dLambda2_dQ2)
f_h = trigsimp(f_h)
f_h

Matrix([
[                 -S*\frac{\partial\Lambda_1}{\partial Q_2}*k*varepsilon*cos(k*x)],
[S*k*varepsilon*(-\frac{\partial\Lambda_2}{\partial Q_2} + alpha - k**2)*cos(k*x)],
[                                                                               0]])

In [19]:
A1, B0, B1 = symbols('A1 B0 B1')

In [20]:
f_h = f_h.subs(dLambda1_dQ2, A1 * sin(k*x))
f_h = f_h.subs(dLambda2_dQ2, B0 + B1 * cos(2*k*x))
f_h = trigsimp(expand(f_h))
f_h = simplify(sincos_to_sum(f_h))
f_h

Matrix([
[                                                                    -A1*S*k*varepsilon*sin(2*k*x)/2],
[S*k*varepsilon*(-2*B0*cos(k*x) - B1*(cos(k*x) + cos(3*k*x)) + 2*alpha*cos(k*x) - 2*k**2*cos(k*x))/2],
[                                                                                                  0]])

In [84]:
f_h_1 = collect(expand(f_h[0]), sin(k*x))
f_h_2 = collect(expand(f_h[1]), (cos(k*x), cos(3*k*x)))

In [98]:
f_hx_2 = -Rational(1, 2) * A1 * S * k * eps

In [93]:
f_hy_1 = f_h_2.args[0].args[0]
f_hy_3 = -Rational(1, 2) * B1 * S * k * eps

In [103]:
(zeta_2 * f_hx_2 / (-2*k)).subs(subs_dict)

-0.000420078190212500

In [104]:
(zeta_2 * f_hy_1 / k**2).subs(subs_dict)

0.00132053995240282

In [105]:
(zeta_2 * f_hy_3 / (9 * k**2)).subs(subs_dict)

-3.52400278419067e-5

## Generate $\mu_1$ stress tensor and corresponding force

In [21]:
sigma_mu1 = simplify(h * Q_mat - Q_mat * h)
sigma_mu1

Matrix([
[                                                                                                                   0, S*(S*k**2*varepsilon*sin(k*x) - varepsilon*\Lambda_1(x)*sin(k*x) + varepsilon*\Lambda_4(x)*sin(k*x) + \Lambda_2(x)), 0],
[S*(-S*k**2*varepsilon*sin(k*x) + varepsilon*\Lambda_1(x)*sin(k*x) - varepsilon*\Lambda_4(x)*sin(k*x) - \Lambda_2(x)),                                                                                                                   0, 0],
[                                                                                                                   0,                                                                                                                   0, 0]])

In [22]:
f_mu1 = zeros(3, 1)
for i in range(3):
    f_mu1[i, 0] = sum([sigma_mu1[k, i].diff(xi[k])
                     for k in range(3)])

f_mu1 = simplify(f_mu1)
f_mu1

Matrix([
[                                                                                                                                                                                                                                         0],
[S*(S*k**3*varepsilon*cos(k*x) - k*varepsilon*\Lambda_1(x)*cos(k*x) + k*varepsilon*\Lambda_4(x)*cos(k*x) - varepsilon*sin(k*x)*Derivative(\Lambda_1(x), x) + varepsilon*sin(k*x)*Derivative(\Lambda_4(x), x) + Derivative(\Lambda_2(x), x))],
[                                                                                                                                                                                                                                         0]])

In [23]:
dLambda4_dQ2 = symbols('\\frac{\partial\Lambda_4}{\partial\ Q_2}')

In [24]:
f_mu1 = f_mu1.subs(Lambda1.diff(x), 
                   Q_vec[1].diff(x) * dLambda1_dQ2)
f_mu1 = f_mu1.subs(Lambda2.diff(x), 
                   Q_vec[1].diff(x) * dLambda2_dQ2)
f_mu1 = f_mu1.subs(Lambda4.diff(x), 
                   Q_vec[1].diff(x) * dLambda4_dQ2)
f_mu1 = trigsimp(f_mu1)
f_mu1

Matrix([
[                                                                                                                                                                                                                                       0],
[S*k*varepsilon*(-S*\frac{\partial\Lambda_1}{\partial Q_2}*varepsilon*sin(k*x) + S*\frac{\partial\Lambda_2}{\partial Q_2} + S*\frac{\partial\Lambda_4}{\partial Q_2}*varepsilon*sin(k*x) + S*k**2 - \Lambda_1(x) + \Lambda_4(x))*cos(k*x)],
[                                                                                                                                                                                                                                       0]])

In [25]:
C1, D0, D1, E0, E1 = symbols('C1 D0 D1 E0 E1')

In [26]:
f_mu1 = f_mu1.subs(dLambda1_dQ2, A1*sin(k*x))
f_mu1 = f_mu1.subs(dLambda2_dQ2, B0 + B1*cos(2*k*x))
f_mu1 = f_mu1.subs(dLambda4_dQ2, C1*sin(k*x))
f_mu1 = f_mu1.subs(Lambda1, D0 + D1*cos(2*k*x))
f_mu1 = f_mu1.subs(Lambda4, E0 + E1*cos(2*k*x))
f_mu1 = simplify(sincos_to_sum(f_mu1))
f_mu1

Matrix([
[                                                                                                                                                                                                                                                                                                                0],
[S*k*varepsilon*(-A1*S*varepsilon*cos(k*x) + A1*S*varepsilon*cos(3*k*x) + 4*B0*S*cos(k*x) + 2*B1*S*cos(k*x) + 2*B1*S*cos(3*k*x) + C1*S*varepsilon*cos(k*x) - C1*S*varepsilon*cos(3*k*x) - 4*D0*cos(k*x) - 2*D1*cos(k*x) - 2*D1*cos(3*k*x) + 4*E0*cos(k*x) + 2*E1*cos(k*x) + 2*E1*cos(3*k*x) + 4*S*k**2*cos(k*x))/4],
[                                                                                                                                                                                                                                                                                                                0]])

In [39]:
f_mu1_expr = collect(f_mu1[1], (cos(k*x), cos(3*k*x)))
f_mu1_expr

S*k*varepsilon*((A1*S*varepsilon + 2*B1*S - C1*S*varepsilon - 2*D1 + 2*E1)*cos(3*k*x) + (-A1*S*varepsilon + 4*B0*S + 2*B1*S + C1*S*varepsilon - 4*D0 - 2*D1 + 4*E0 + 2*E1 + 4*S*k**2)*cos(k*x))/4

In [51]:
c_mu1_1 = f_mu1_expr.args[4].args[1].args[0] * S*k*eps
c_mu1_1

S*k*varepsilon*(-A1*S*varepsilon + 4*B0*S + 2*B1*S + C1*S*varepsilon - 4*D0 - 2*D1 + 4*E0 + 2*E1 + 4*S*k**2)

In [52]:
c_mu1_3 = f_mu1_expr.args[4].args[0].args[0] * S*k*eps
c_mu1_3

S*k*varepsilon*(A1*S*varepsilon + 2*B1*S - C1*S*varepsilon - 2*D1 + 2*E1)

In [53]:
print(latex(c_mu1_1))

S k \varepsilon \left(- A_{1} S \varepsilon + 4 B_{0} S + 2 B_{1} S + C_{1} S \varepsilon - 4 D_{0} - 2 D_{1} + 4 E_{0} + 2 E_{1} + 4 S k^{2}\right)


In [54]:
print(latex(c_mu1_3))

S k \varepsilon \left(A_{1} S \varepsilon + 2 B_{1} S - C_{1} S \varepsilon - 2 D_{1} + 2 E_{1}\right)


In [58]:
zeta_1 = symbols('\zeta_1')
zeta_1

\zeta_1

In [101]:
zeta_2 = symbols('\zeta_2')
zeta_2

\zeta_2

In [102]:
subs_dict = {S : 0.6751, k : 0.0810, eps : 0.01,
             A1 : 0.1385, B0 : 8.004373, B1 : -0.004235,
             C1 : -0.4263, D0 : 3.6008289, D1 : -0.0002337,
             E0 : -1.8010167, E1 : 0.0007191, zeta_1 : 1.0050,
             alpha : 8.0, zeta_2 : -1.7971}

In [106]:
(c_mu1_1 * zeta_1 / k**2).subs(subs_dict)

0.00148412845156261

In [107]:
(c_mu1_3 * zeta_1 / (9 * k**2)).subs(subs_dict)

4.35378379835324e-9